In [3]:
!pip install unstructured pytesseract Pillow pymupdf pdfminer.six unstructured-inference pi-heif pdf2image unstructured_pytesseract


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
!apt-get update

In [1]:
!apt-get -y install tesseract-ocr ffmpeg libsm6 libxext6 poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsm6 is already the newest version (2:1.2.3-1build2).
libxext6 is already the newest version (2:1.3.4-1build1).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  liblcms2-2 libnspr4 libnss3 libpoppler118 poppler-data
Suggested packages:
  liblcms2-utils ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  liblcms2-2 libnspr4 libnss3 libpoppler118 poppler-data poppler-utils
0 upgraded, 6 newly installed, 0 to remove and 31 not upgraded.
Need to get 5053 kB of archives.
After this operation, 22.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 poppler-data all 0.4

In [8]:
import os
import hashlib
from PIL import Image
import pytesseract
from unstructured.partition.pdf import partition_pdf
import fitz  # PyMuPDF

def calculate_hash(file_path):
    """Вычисляет SHA-256 хеш файла."""
    sha256 = hashlib.sha256()
    with open(file_path, 'rb') as f:
        while chunk := f.read(8192):
            sha256.update(chunk)
    return sha256.hexdigest()

def pdf_to_text_old(file_path):
    """Конвертирует PDF файл в текст."""
    doc = fitz.open(file_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

def pdf_to_text(file_path):
    """Конвертирует PDF файл в текст с использованием unstructured и OCR."""
    elements = partition_pdf(file_path,strategy="hi_res")
    text = ""
    for element in elements:
        if element.category == "Image":
            # Преобразуем изображение в текст с помощью OCR
            #image = Image.open(element.metadata.image_path)
            #text += pytesseract.image_to_string(image)
            pass
        else:
            text += element.text + '\n'
    return text

def process_pdf_folder(folder_path, output_folder):
    """Обрабатывает папку с PDF файлами, конвертирует их в текст и исключает дубликаты."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    seen_hashes = set()
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            file_path = os.path.join(folder_path, filename)
            file_hash = calculate_hash(file_path)

            if file_hash not in seen_hashes:
                seen_hashes.add(file_hash)
                text = pdf_to_text(file_path)
                output_file_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")
                with open(output_file_path, 'w', encoding='utf-8') as output_file:
                    output_file.write(text)
                print(f"Processed: {filename}")
            else:
                print(f"Duplicate found, skipping: {filename}")



In [9]:
# Пример использования
folder_path = 'pdf_files'
output_folder = 'txt_files2'
process_pdf_folder(folder_path, output_folder)

Processed: 1.pdf
Processed: 285 (1).pdf
Duplicate found, skipping: 285.pdf
Processed: AD (1).pdf
Duplicate found, skipping: AD.pdf
Processed: ae1983.pdf
Processed: amadeus-3.pdf
Processed: bb_class.pdf
Processed: brega_reliability_in_real-time_why_strong-typed_programming_languages_matter.pdf
Processed: burrows_arm_embedded_development_using_oberon-07.pdf
Processed: campbell_industrial_experience_with_an_engineering_perspective.pdf
Processed: chapuis_oberon-based_autopilots_for_unmanned_aerial_vehicles.pdf
Processed: computersystemdesign (1).pdf
Duplicate found, skipping: ComputerSystemDesign.pdf
Processed: crelier_r.eth_technical_report_125.pdf
Processed: crelier_r.op2_a_portable_oberon_2_compiler.en.pdf
Processed: c_pfister_component_software.pdf
Processed: denisov_razrabotka_komponentnogo_programmnogo_obespecheniya_dlya_nauchnogo_modelirovaniya_i_inzhenerii.pdf
Processed: designideas.pdf
Processed: design_patterns_in_oberon-2_and_component_pascal_c9_ru.pdf
Processed: developinginbla

KeyboardInterrupt: 